In [73]:
import numpy as np
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate
from tensorflow.keras.optimizers import Adam

%matplotlib inline

In [157]:
files = os.listdir("ultrasound-nerve-segmentation/train")
image_names = []
for f in files:
    if '_mask' not in f:
        image_names.append(os.path.splitext(f)[0])

In [158]:
img_width = 128
img_height = 128
batch_size = 32
learning_rate = 1e-5

In [159]:
# a smoothed version of dice coefficient
def dice(y, y_pred):
    intersection = K.sum(K.flatten(y) * K.flatten(y_pred))
    return (2.0 * intersection + 1) / (K.sum(y) + K.sum(y_pred) + 1)

def dice_loss(y, y_pred):
    return -tf.log(dice(y, y_pred))

In [160]:
# Build U-Net/FCN style model
def build_net(img_width, img_height, batch_size, learning_rate):
    input_images = Input(shape=(img_width,img_height,1),batch_size=batch_size, name='input')
    conv1 = Conv2D(32, (3,3), activation='relu', padding='same')(input_images)
    conv1 = Conv2D(32, (3,3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(2)(conv1)

    conv2 = Conv2D(64, (3,3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3,3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(2)(conv2)

    conv3 = Conv2D(128, (3,3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3,3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(2)(conv3)

    conv4 = Conv2D(256, (3,3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3,3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(2)(conv4)

    conv5 = Conv2D(512, (3,3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3,3), activation='relu', padding='same')(conv5)

    conv6 = concatenate([Conv2DTranspose(256, (2,2), strides=(2,2), padding='same')(conv5), conv4])
    conv6 = Conv2D(256, (3,3), activation='relu', padding='same')(conv6)
    conv6 = Conv2D(256, (3,3), activation='relu', padding='same')(conv6)

    conv7 = concatenate([Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(conv6), conv3])
    conv7 = Conv2D(128, (3,3), activation='relu', padding='same')(conv7)
    conv7 = Conv2D(128, (3,3), activation='relu', padding='same')(conv7)

    conv8 = concatenate([Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(conv7), conv2])
    conv8 = Conv2D(64, (3,3), activation='relu', padding='same')(conv8)
    conv8 = Conv2D(64, (3,3), activation='relu', padding='same')(conv8)

    conv9 = concatenate([Conv2DTranspose(32, (2,2), strides=(2,2), padding='same')(conv8), conv1])
    conv9 = Conv2D(32, (3,3), activation='relu', padding='same')(conv9)
    conv9 = Conv2D(32, (3,3), activation='relu', padding='same')(conv9)

    output = Conv2D(1, (1,1), activation='sigmoid')(conv9)

    model = Model(inputs=[input_images], outputs=[output])
    model.compile(optimizer=Adam(lr=learning_rate), loss=dice_loss, metrics=[dice])
    
    return model

In [161]:
def preprocess(im):
    im_arr = np.asarray(im.resize((img_width, img_height)))
    return im_arr

def augment(img):
    #flip h
    #flip v
    #rotate
    pass
    

In [162]:
def get_batches(batch_size):
    while True:
        for i in range(0, len(train_names), batch_size):
            images = []
            labels = []
            for train_name in train_names[i:i+batch_size]:
                im = Image.open("ultrasound-nerve-segmentation/train/"+train_name+".tif")
                lb = Image.open("ultrasound-nerve-segmentation/train/"+train_name+"_mask.tif")

                im_arr = preprocess(im)
                lb_arr = preprocess(lb)
    #             augment(im_arr)
                images.append(np.expand_dims(im_arr, axis=2))
                labels.append(np.expand_dims(lb_arr, axis=2))

            ims = np.array(images, dtype='float32')
            lbls = np.array(labels)

            mean = np.mean(ims)
            std = np.std(ims)
            ims -= mean
            ims /= std
            lbls = lbls.astype('float32')
            lbls /= 255.0

            yield ims, lbls

In [163]:
import math
def train(epochs):
    
    gen = get_batches(batch_size)
    
    model = build_net(img_width, img_height, batch_size, learning_rate)
    checkpoint = ModelCheckpoint('model_weights.hd5', monitor='val_loss')
    model.fit_generator(gen, epochs=epochs, steps_per_epoch=math.ceil(5071/batch_size),
                        verbose=1, callbacks=[checkpoint])
    

In [164]:
train(50)

Epoch 1/50
10/10 [==============================] - 47s 5s/step - loss: 3.8377 - dice: 0.0229
Epoch 2/50
10/10 [==============================] - 20s 2s/step - loss: 3.8351 - dice: 0.0230
Epoch 3/50
10/10 [==============================] - 22s 2s/step - loss: 3.8325 - dice: 0.0231
Epoch 4/50
10/10 [==============================] - 21s 2s/step - loss: 3.8300 - dice: 0.0231
Epoch 5/50
10/10 [==============================] - 21s 2s/step - loss: 3.8275 - dice: 0.0232
Epoch 6/50
10/10 [==============================] - 20s 2s/step - loss: 3.8250 - dice: 0.0232
Epoch 7/50
10/10 [==============================] - 20s 2s/step - loss: 3.8224 - dice: 0.0233
Epoch 8/50
10/10 [==============================] - 20s 2s/step - loss: 3.8197 - dice: 0.0234
Epoch 9/50
10/10 [==============================] - 21s 2s/step - loss: 3.8167 - dice: 0.0234
Epoch 10/50
10/10 [==============================] - 21s 2s/step - loss: 3.8135 - dice: 0.0235
Epoch 11/50
10/10 [==============================] - 22s 2s